![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_AlbertForMultipleChoice.ipynb)

## Import ONNX AlbertForMultipleChoice models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `AlbertForMultipleChoice` is only available since in `Spark NLP 5.6.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import ALBERT models trained/fine-tuned for question answering via `AlbertForMultipleChoice` or `AlbertForMultipleChoice`. These models are usually under `Multiple Choice` category and have `bert` in their labels
- Reference: [AlbertForMultipleChoice](https://huggingface.co/docs/transformers/main/en/model_doc/albert#transformers.AlbertForMultipleChoice)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers[onnx]==4.48.2 optimum onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 880.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [albert-base-v2](https://huggingface.co/albert-base-v2 ) model from HuggingFace as an example and load it as a `ORTModelForMultipleChoice`, representing an ONNX model.
- In addition to the ALBERT model, we also need to save the `AlbertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForMultipleChoice

MODEL_NAME = "Ariffiq99/CRAB_COPA_KUCI_e_care_albert_Base_Finetuned"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForMultipleChoice.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

('onnx_models/Ariffiq99/CRAB_COPA_KUCI_e_care_albert_Base_Finetuned/tokenizer_config.json',
 'onnx_models/Ariffiq99/CRAB_COPA_KUCI_e_care_albert_Base_Finetuned/special_tokens_map.json',
 'onnx_models/Ariffiq99/CRAB_COPA_KUCI_e_care_albert_Base_Finetuned/spiece.model',
 'onnx_models/Ariffiq99/CRAB_COPA_KUCI_e_care_albert_Base_Finetuned/added_tokens.json',
 'onnx_models/Ariffiq99/CRAB_COPA_KUCI_e_care_albert_Base_Finetuned/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -lh {EXPORT_PATH}

total 48M
-rw-r--r-- 1 root root  871 Jun 25 10:23 config.json
-rw-r--r-- 1 root root  45M Jun 25 10:23 model.onnx
-rw-r--r-- 1 root root  970 Jun 25 10:23 special_tokens_map.json
-rw-r--r-- 1 root root 743K Jun 25 10:23 spiece.model
-rw-r--r-- 1 root root 1.5K Jun 25 10:23 tokenizer_config.json
-rw-r--r-- 1 root root 2.2M Jun 25 10:23 tokenizer.json


We need the `spiece.model` for the Tokenizer. This is the same for every model, these are assets (saved in /assets) needed for tokenization inside Spark NLP.

In [4]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/spiece.model {EXPORT_PATH}/assets/

In [5]:
!ls -lR {EXPORT_PATH}

onnx_models/Ariffiq99/CRAB_COPA_KUCI_e_care_albert_Base_Finetuned:
total 48312
drwxr-xr-x 2 root root     4096 Jun 25 10:23 assets
-rw-r--r-- 1 root root      871 Jun 25 10:23 config.json
-rw-r--r-- 1 root root 47180962 Jun 25 10:23 model.onnx
-rw-r--r-- 1 root root      970 Jun 25 10:23 special_tokens_map.json
-rw-r--r-- 1 root root     1442 Jun 25 10:23 tokenizer_config.json
-rw-r--r-- 1 root root  2272611 Jun 25 10:23 tokenizer.json

onnx_models/Ariffiq99/CRAB_COPA_KUCI_e_care_albert_Base_Finetuned/assets:
total 744
-rw-r--r-- 1 root root 760289 Jun 25 10:23 spiece.model


Voila! We have our vocab.txt inside assets directory

## Import and Save AlbertForMultipleChoice in Spark NLP

- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [6]:
!pip install -q spark-nlp==6.0.3 pyspark==3.5.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.0/713.0 kB 24.6 MB/s eta 0:00:00


In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  6.0.3
Apache Spark version:  3.5.1


- Let's use `loadSavedModel` functon in `AlbertForMultipleChoice` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `AlbertForMultipleChoice` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [8]:
from sparknlp.annotator import AlbertForMultipleChoice

albertMultpleChoiceClassifier = AlbertForMultipleChoice.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document_question", "document_context"])\
  .setOutputCol("answer")

Let's save it on disk so it is easier to be moved around and also be used later via .load function

In [9]:
albertMultpleChoiceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny `AlbertForMultipleChoice` model in Spark NLP 🚀 pipeline!

In [11]:
data = spark.createDataFrame([
    ("In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.",
     "It is eaten with a fork and a knife, It is eaten while held in the hand."),
    ("The Eiffel Tower is located in which country?", "Germany, France, Italy"),
    ("Which animal is known as the king of the jungle?", "Lion, Elephant, Tiger, Leopard"),
    ("Water boils at what temperature?", "90°C, 120°C, 100°C"),
    ("Which planet is known as the Red Planet?", "Jupiter, Mars, Venus"),
    ("Which language is primarily spoken in Brazil?", "Spanish, Portuguese, English"),
    ("The Great Wall of China was built to protect against invasions from which group?",
     "The Greeks, The Romans, The Mongols, The Persians"),
    ("Which chemical element has the symbol 'O'?", "Oxygenm, Osmium, Ozone"),
    ("Which continent is the Sahara Desert located in?", "Asia, Africa, South America"),
    ("Which artist painted the Mona Lisa?", "Vincent van Gogh, Leonardo da Vinci, Pablo Picasso")
], ["question", "choices"])

data.show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|question                                                                                  |choices                                                                 |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |
|Which animal is known as the king of the jungle?                                          |Lion, Elephant, Tiger, Leopard                                          |
|Wat

In [12]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import AlbertForMultipleChoice
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "choices"]) \
    .setOutputCols(["document_question", "document_choices"])

albert_for_multiple_choice = AlbertForMultipleChoice() \
    .load(f"./{MODEL_NAME}_spark_nlp_onnx") \
    .setInputCols(["document_question", "document_choices"]) \
    .setOutputCol("answer") \
    .setBatchSize(4)

pipeline = Pipeline(stages=[
    document_assembler,
    albert_for_multiple_choice
])

model = pipeline.fit(data)
results = model.transform(data)

results.select("question", "choices", "answer.result").show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+--------------------------------------+
|question                                                                                  |choices                                                                 |result                                |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+--------------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|[ It is eaten while held in the hand.]|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |[ Italy]       